<a href="https://colab.research.google.com/github/AbdurNawaz/Charc-RNN/blob/master/Charac_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf

In [0]:
with open('anna.txt') as f:
    text = f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [6]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [7]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

In [0]:
def get_batches(arr, batch_size, n_steps):
  char_per_batch = batch_size*n_steps
  num_batches = len(arr)//char_per_batch

  arr = arr[:num_batches*char_per_batch]
  arr = arr.reshape((batch_size, -1))

  for i in range(0, arr.shape[1], n_steps):
    x = arr[:, i:i+n_steps]
    y_temp = arr[:, i+1:i+n_steps+1]

    y = np.zeros(x.shape, dtype=x.dtype)
    y[:, :y_temp.shape[1]] = y_temp

    yield x, y

In [9]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:, :10])
print('y\n', y[:, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]
y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


In [0]:
def build_inputs(batch_size, n_steps):
  inputs = tf.placeholder(tf.int32, [batch_size, n_steps], 'inputs')
  targets = tf.placeholder(tf.int32, [batch_size, n_steps], 'targets')

  keep_prob = tf.placeholder(tf.float32, name='keep_prob')

  return inputs, targets, keep_prob


In [0]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):

  def build_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

    return drop
  
  cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
  initial_state = cell.zero_state(batch_size, tf.float32)

  return cell, inital_state


In [0]:
def build_output(lstm_output, in_size, out_size):
  seq_output = tf.concat(lstm_output, axis=1)
  x = tf.reshape(seq_output, [-1, in_size])

  with tf.variable_scope('softmax'):
    softmax_w = tf.Variable(tf.truncated_normal((insize, out_size), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(out_size))

  logits = tf.matmul(x, softmax_w) + softmax_b

  out = tf.nn.softmax(logits, name='predictions')

  return out, logits

In [0]:
def build_loss(logits, targets, lstm_size, num_classes):

  y_one_hot = tf.one_hot(targets, num_classes)
  y_reshaped = tf.reshaped(y_one_hot, logits.shape)

  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
  loss = tf.reduce_mean(loss)

  return loss

In [0]:
#Clipping Gradients to avoid exploding

def build_optimizer(loss, learning_rate, grad_clip):
  tvars = tf.trainalbe_variables()
  grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
  train_op = tf.train.AdamOptimizer(leaning_rate)
  optimizer = train_op.apply_gradients(zip(grads, tvars))

  return optimizer

In [0]:
class charRNN:

  def __init__(self, num_classes, batch_size=64, num_steps=50, lstm_size=128,
               num_layers=2, learning_rate=0.001, grad_clip=5, sampling=False):
    if sampling == True:
      batch_szie, num_steps = 1, 1
    else:
      batch_size, num_steps = batch_size, num_steps

    tf.reset_default_graph()

    self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
    cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

    x_one_hot = tf.one_hot(self.inputs, num_classes)

    outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
    self.final_state = state 
    self.predication, self.logits = build_output(outputs,lstm_size, num_classes)

    self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
    self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
